In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

In [2]:
# Load data
external_df = pd.read_csv("Data_External.csv")
internal_df = pd.read_csv("Data_Internal.csv")

In [3]:
external_df.head(5)

,PRODUCT_NAME,UNIT_OF_MEASURE
0,5 HOUR XTRA GRAPE 1.93 OZ,1
1,B - PB & HONEY SAMMICH,1
2,B - RUDY FARMS - SAUSAGE AND BISCUIT TWIN,1
3,BANANAS - FRESH,1
4,BOBOS PB&J GRAPE 2.1 OZ,1


In [4]:
internal_df.head(5)

,NAME,OCS_NAME,LONG_NAME
0,3 Mskt DkChocMnt 1.24oz,3 Mskt DkChocMnt 1.24oz,3 Musketeers Dark Chocolate Mint (1.24oz)
1,NaN,Costco Choc Mini 4.69lb,Costco Chocolate Mini (4.69lb)
2,Dove Dk Choc Bars 1.3oz,Dove Dk Choc Bars 1.3oz,Dove Dark Chocolate Bars (1.3oz)
3,Fishers ChocPnut 3.5oz,Fishers ChocPnut 3.5oz,Fisher's Chocolate Peanuts (3.5oz)
4,HariboGummiGoldBear2oz,HariboGummiGoldBear2oz,Haribo Gummi Gold-Bears (2oz)


In [5]:
external_df.shape

(43, 2)

In [6]:
internal_df.shape

(16192, 3)

In [7]:
external_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT_NAME     43 non-null     object
 1   UNIT_OF_MEASURE  43 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 816.0+ bytes


In [8]:
internal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16192 entries, 0 to 16191
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   NAME       7454 non-null   object
 1   OCS_NAME   9292 non-null   object
 2   LONG_NAME  16192 non-null  object
dtypes: object(3)
memory usage: 379.6+ KB


In [9]:
# Combine all product names for vectorization
all_products = list(external_df['PRODUCT_NAME']) + list(internal_df['LONG_NAME'])

In [10]:
# Convert product names into TF-IDF vectors
vectorizer = TfidfVectorizer().fit(all_products)
tfidf_matrix = vectorizer.transform(all_products)

In [11]:
# Split back into separate vectors
external_vectors = tfidf_matrix[:len(external_df)]
internal_vectors = tfidf_matrix[len(external_df):]

# Compute cosine similarity between each external and internal product
similarity_matrix = cosine_similarity(external_vectors, internal_vectors)

# Set similarity threshold for a match
SIMILARITY_THRESHOLD = 0.75


In [12]:
# Determine true matches
y_true = [
    1 if similarity_matrix[i].max() > SIMILARITY_THRESHOLD else 0 
    for i in range(len(external_df))
]

# Find best internal match for each external product
y_pred = []
matched_products = []
for i in range(len(external_df)):
    best_match_idx = similarity_matrix[i].argmax()
    best_match_score = similarity_matrix[i].max()

    if best_match_score > SIMILARITY_THRESHOLD:
        y_pred.append(1)
        matched_products.append(internal_df.iloc[best_match_idx]['LONG_NAME'])
    else:
        y_pred.append(0)
        matched_products.append("NULL")

# Add results to external_df for verification
external_df['matched_internal_product'] = matched_products
external_df['similarity_score'] = [similarity_matrix[i].max() for i in range(len(external_df))]

In [13]:
print(f"Threshold Value: {SIMILARITY_THRESHOLD}")
print("*"*40)
print(f"Total Products: {len(y_true)}, True Matches Found: {sum(y_true)}, Predicted Matches: {sum(y_pred)}")
print("Precision:", precision_score(y_true, y_pred, zero_division=1))
print("Recall:", recall_score(y_true, y_pred, zero_division=1))
print("F1-Score:", f1_score(y_true, y_pred, zero_division=1))

# Display results
external_df[['PRODUCT_NAME', 'matched_internal_product', 'similarity_score']]


Threshold Value: 0.75
****************************************
Total Products: 43, True Matches Found: 3, Predicted Matches: 3
Precision: 1.0
Recall: 1.0
F1-Score: 1.0


,PRODUCT_NAME,matched_internal_product,similarity_score
0,5 HOUR XTRA GRAPE 1.93 OZ,NULL,0.412106
1,B - PB & HONEY SAMMICH,NULL,0.719995
2,B - RUDY FARMS - SAUSAGE AND BISCUIT TWIN,NULL,0.442868
3,BANANAS - FRESH,NULL,0.490718
4,BOBOS PB&J GRAPE 2.1 OZ,NULL,0.336797
5,BODY ARMOR STRWBRY BANANA 16 OZ,NULL,0.217805
6,BR ESPRESSO W/ CREAM 11 OZ,NULL,0.311721
7,Bumble Bee Tuna Salad 3.5oz,Bumble Bee Tuna Salad with Crackers (3.5oz),0.859730
8,CELSIUS ORANGE ENERGY 12 OZ,NULL,0.382441
9,CELSIUS PEACH VIBE 12 OZ,NULL,0.721988


In [14]:
# # Convert result to Dataframe

# Save results to CSV file
external_df.to_csv("matched_products.csv", index=False)
print("File saved as matched_products.csv")

File saved as matched_products.csv
